In [1]:
import requests
import json
import pandas as pd

In [2]:
res = requests.get("https://cdn-api.co-vin.in/api/v2/admin/location/states")

In [3]:
states = pd.DataFrame(json.loads(res.text)["states"])

In [4]:
states.to_csv("../data/states.csv", index = False) #sates with id 

In [5]:
df_states = pd.read_csv("../data/states.csv")

In [6]:
df_states.head()

,state_id,state_name
0,1,Andaman and Nicobar Islands
1,2,Andhra Pradesh
2,3,Arunachal Pradesh
3,4,Assam
4,5,Bihar


In [7]:
df_district_all = None
for state_id in df_states["state_id"].values.tolist():
    state_name = df_states.loc[df_states['state_id'] == state_id, "state_name"].item()
    res = requests.get("https://cdn-api.co-vin.in/api/v2/admin/location/districts/{}".format(state_id))
    df_district = pd.DataFrame(json.loads(res.text)["districts"])
    df_district["state_name"] = state_name
    if df_district_all is None:
        df_district_all = df_district
    else:
        df_district_all = pd.concat([df_district_all, df_district], ignore_index=True)

df_district_all = df_district_all.sort_values("district_name")
df_district_all.to_csv("../data/districts.csv", index=False)    #District_id

In [8]:
df_district_all.head()

,district_id,district_name,state_name
598,582,Adilabad,Telangana
326,320,Agar,Madhya Pradesh
324,796,Agatti Island,Lakshadweep
639,622,Agra,Uttar Pradesh
158,154,Ahmedabad,Gujarat


## Search by district id 

In [9]:
#Example

district_id = "240" #Ranchi 
date_from = "03-05-2021"

In [10]:
URL = "https://cdn-api.co-vin.in/api/v2/appointment/sessions/public/calendarByDistrict?district_id={}&date={}" \
      .format(district_id,date_from)
res = requests.get(URL)

In [11]:
calender_df = pd.DataFrame(json.loads(res.text)["centers"])
calender_df.head()

,center_id,name,state_name,district_name,block_name,pincode,lat,long,from,to,fee_type,sessions,vaccine_fees
0,607446,SEVA SADAN,Jharkhand,Ranchi,Ranchi Urban,834001,0,0,10:00:00,18:00:00,Paid,[{'session_id': '25c9ec55-0a30-41f0-a5e6-f8c32...,NaN
1,573818,Angara CHC,Jharkhand,Ranchi,Angara,834006,23,85,09:00:00,17:00:00,Free,[{'session_id': '7fa9402b-4d6d-4358-a8a4-cf6c5...,NaN
2,691192,NEW BIDHAN SABHA,Jharkhand,Ranchi,Ranchi Urban,834004,0,0,09:00:00,17:00:00,Free,[{'session_id': 'a2061f11-fc90-4957-85a2-cf6d2...,NaN
3,577815,LAPUNG CHC,Jharkhand,Ranchi,Lapung,835234,23,84,09:00:00,17:00:00,Free,[{'session_id': 'c1f95fba-8ebc-45ec-ba28-1b480...,NaN
4,577785,CHANHO CHC,Jharkhand,Ranchi,Chanho,835214,23,84,09:00:00,17:00:00,Free,[{'session_id': '355742c4-9294-4bf7-b95b-7230c...,NaN


In [12]:
calender_df = calender_df[['name', 'state_name', 'district_name',
       'pincode', 'sessions','fee_type','vaccine_fees']]
calender_df.head()

,name,state_name,district_name,pincode,sessions,fee_type,vaccine_fees
0,SEVA SADAN,Jharkhand,Ranchi,834001,[{'session_id': '25c9ec55-0a30-41f0-a5e6-f8c32...,Paid,NaN
1,Angara CHC,Jharkhand,Ranchi,834006,[{'session_id': '7fa9402b-4d6d-4358-a8a4-cf6c5...,Free,NaN
2,NEW BIDHAN SABHA,Jharkhand,Ranchi,834004,[{'session_id': 'a2061f11-fc90-4957-85a2-cf6d2...,Free,NaN
3,LAPUNG CHC,Jharkhand,Ranchi,835234,[{'session_id': 'c1f95fba-8ebc-45ec-ba28-1b480...,Free,NaN
4,CHANHO CHC,Jharkhand,Ranchi,835214,[{'session_id': '355742c4-9294-4bf7-b95b-7230c...,Free,NaN


In [13]:
df =calender_df.explode("sessions")
df['min_age_limit'] = df.sessions.apply(lambda x: x['min_age_limit'])
df['available_capacity'] = df.sessions.apply(lambda x: x['available_capacity'])
df['date'] = df.sessions.apply(lambda x: x['date'])

In [14]:
df.head()

,name,state_name,district_name,pincode,sessions,fee_type,vaccine_fees,min_age_limit,available_capacity,date
0,SEVA SADAN,Jharkhand,Ranchi,834001,{'session_id': '25c9ec55-0a30-41f0-a5e6-f8c320...,Paid,NaN,45,0,03-05-2021
0,SEVA SADAN,Jharkhand,Ranchi,834001,{'session_id': 'f54258a7-de3e-4dc2-913b-250a71...,Paid,NaN,45,0,04-05-2021
0,SEVA SADAN,Jharkhand,Ranchi,834001,{'session_id': '50575e95-2c30-4a05-8e7c-238fe3...,Paid,NaN,45,0,05-05-2021
0,SEVA SADAN,Jharkhand,Ranchi,834001,{'session_id': '0acc546d-ad81-43eb-998e-95fa11...,Paid,NaN,45,100,06-05-2021
0,SEVA SADAN,Jharkhand,Ranchi,834001,{'session_id': '10f428a1-dc9d-48d8-9192-604592...,Paid,NaN,45,100,07-05-2021


In [15]:
df = df[['date','available_capacity','min_age_limit', 'name', 'state_name', 'district_name',
       'pincode', 'fee_type','vaccine_fees']]
df = df[df["pincode"] == 834005]
df

,date,available_capacity,min_age_limit,name,state_name,district_name,pincode,fee_type,vaccine_fees
37,03-05-2021,11,45,GURUDWARA PISKA MORE,Jharkhand,Ranchi,834005,Free,NaN
37,04-05-2021,22,45,GURUDWARA PISKA MORE,Jharkhand,Ranchi,834005,Free,NaN
37,05-05-2021,31,45,GURUDWARA PISKA MORE,Jharkhand,Ranchi,834005,Free,NaN
